## Connect to Drive and importing dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
pip install swifter

In [ ]:
import pandas as pd
import numpy as np
import swifter
import re
from sklearn.model_selection import KFold
import statistics
import warnings
import seaborn as sns
from tqdm import tqdm
warnings.filterwarnings("ignore")

pd.options.mode.chained_assignment = None

## Read the data

In [ ]:
pathTrain = "/content/drive/MyDrive/NLP Study Group/Problem 4/train.dep.txt"
pathTest = "/content/drive/MyDrive/NLP Study Group/Problem 4/test.dep.txt"

In [ ]:
columns = ['index','word', 'pos', 'pos1', 'unused', 'parent', 'type']

In [ ]:
train = pd.read_csv(pathTrain, sep = "\t", names = columns)
train = train[['word', 'pos', 'parent', 'type']]
train.reset_index(inplace = True)
train

,index,word,pos,parent,type
0,1,in,IN,43,PP
1,2,an,DT,5,DEP
2,3,oct.,NNP,5,DEP
3,4,19,CD,5,DEP
4,5,review,NN,1,NP
...,...,...,...,...,...
5275,4,",",",",7,DEP
5276,5,'','',7,DEP
5277,6,he,PRP,7,NP-SBJ
5278,7,says,VBZ,0,ROOT


In [ ]:
test = pd.read_csv(pathTest, sep = "\t", names = columns)
test = test[['word', 'pos', 'parent', 'type']]
test.reset_index(inplace = True)
test

,index,word,pos,parent,type
0,1,no,UH,4,INTJ
1,2,",",",",4,DEP
2,3,it,PRP,4,NP-SBJ
3,4,was,VBD,0,ROOT
4,5,n't,RB,4,DEP
...,...,...,...,...,...
4634,19,by,IN,6,PP
4635,20,year,NN,21,DEP
4636,21,'s,POS,22,NP
4637,22,end,NN,19,NP


## Class and Functions

In [ ]:
def dataFramePerSentence(dframe, numbSentence):
  tempList = []
  startPoint = 0

  for i in range(numbSentence):
    temp = dframe[startPoint:].copy()  
    temp.reset_index(inplace = True)
    temp['level_0'] = temp['level_0'] - temp['level_0'][0] + 1
    temp = temp[temp['level_0'] == temp['index']]
    startPoint = startPoint + len(temp)
    temp.drop(['level_0', 'index'], axis=1, inplace = True)

    temp.parent = temp.parent - 1

    for index, row in temp.iterrows():
      if row.parent is -1: 
        temp['parent'][index] = 'ROOT'
      elif not isinstance(temp['parent'][index], str):
        key = temp['parent'][index]
        temp['parent'][index] = temp['word'][key]

    tempList.append(temp) 

  return tempList

In [ ]:
class Perceptron:
  def __init__(self):
    self._weights = []
    self._learningRate = 0.01
    self._epoch = 10
    self._x = []
    self._y = []  

  def linearFunction(self, data, weights):
    return np.dot(data, weights)
  
  def activationFunction(self, linearResult):
    return 1 / ( 1 + np.exp(-(linearResult)))

In [ ]:
class ShiftReduce:
  def __init__(self):
    self._sentence = None
    self._stack = None

  def process(self, sentence, model):
    self.sentence = sentence
    self._stack = ['ROOT']

    while len(self._sentence) > 0 and len(self._stack) > 1:
      # if model.predict() == 'shift'
      #   shift()
      # elif model.predict() == 'left'
      #   left()
      # elif model.predict() == 'right'
      #   right()

  #def shift(self)
  #def left(self)
  #def right(self)

## Get dataframe for every sentence

In [ ]:
dfTrainList = dataFramePerSentence(train, 200)
dfTestList = dataFramePerSentence(test, 200)

In [ ]:
dfTrainList[1]

,word,pos,parent,type
0,ms.,NNP,haag,DEP
1,haag,NNP,plays,NP-SBJ
2,plays,VBZ,ROOT,ROOT
3,elianti,NNP,plays,NP-OBJ
4,.,.,plays,DEP


In [ ]:
dfTestList[0]

,word,pos,parent,type
0,no,UH,was,INTJ
1,",",",",was,DEP
2,it,PRP,was,NP-SBJ
3,was,VBD,ROOT,ROOT
4,n't,RB,was,DEP
5,black,JJ,monday,DEP
6,monday,NNP,was,NP-PRD
7,.,.,was,DEP
